# Multi-Ticker First Bar Trading Strategy

## Overview

This notebook implements a **flexible First Bar Strategy** that can be easily applied to different tickers (TQQQ, QQQ, SPY, etc.) using real-time data from **Interactive Brokers TWS API**.

### Key Features:
- **One-variable ticker switching**: Change symbol with single configuration
- **IBKR TWS integration**: Live historical data retrieval
- **Flexible configuration**: Easy parameter adjustment
- **Comprehensive analysis**: Enhanced performance metrics and visualizations
- **Multi-ticker comparison**: Easy switching between different assets

### Strategy Logic:
- **Buy Signal**: First hourly bar closes above its open (bullish momentum)
- **Sell Signal**: First hourly bar closes below its open (bearish momentum)
- **Entry**: At close of first bar (15:30 ET)
- **Exit**: Profit target OR end-of-day close

---

## 🔧 Configuration Section

**Change the ticker symbol below to test different assets:**

In [ ]:
# ===================================================================
# CONFIGURATION SECTION - CHANGE TICKER HERE
# ===================================================================

# === PRIMARY TICKER CONFIGURATION ===
TICKER_SYMBOL = "TQQQ"    # ⬅️ CHANGE THIS TO TEST DIFFERENT TICKERS
                          # Examples: "QQQ", "TQQQ", "SPY", "IWM", "DIA"

# === IBKR CONNECTION SETTINGS ===
TWS_HOST = "127.0.0.1"    # Local TWS/Gateway
TWS_PORT = 7497           # 7497 = Paper Trading, 7496 = Live Trading
CLIENT_ID = 1             # Unique client ID

# === DATA RETRIEVAL SETTINGS ===
EXCHANGE = "SMART"         # Smart routing
CURRENCY = "USD"          # Currency
DATA_DURATION = "2 Y"     # How far back: "1 Y", "2 Y", "5 Y", "10 Y"
BAR_SIZE = "1 hour"       # Bar size for strategy
WHAT_TO_SHOW = "TRADES"   # Data type: "TRADES", "MIDPOINT", "BID", "ASK"

# === STRATEGY PARAMETERS ===
TRADE_MODE = "both"       # "buy", "sell", "both"
PROFIT_TARGET = 2.2       # Profit target in % (optimized from QQQ analysis)
SMA_PERIOD = 0            # SMA filter period (0 = disabled)

# === FALLBACK DATA OPTION ===
USE_FALLBACK_DATA = True  # Use CSV if IBKR connection fails
FALLBACK_CSV_PATH = "qqqh1.csv"  # Fallback data file

print(f"🎯 Configuration loaded for: {TICKER_SYMBOL}")
print(f"📊 Strategy Mode: {TRADE_MODE} | Profit Target: {PROFIT_TARGET}%")
print(f"🔌 IBKR Connection: {TWS_HOST}:{TWS_PORT} (Client ID: {CLIENT_ID})")
print(f"📈 Data Period: {DATA_DURATION} of {BAR_SIZE} bars")

## 📚 Imports and Setup

In [ ]:
# === IMPORTS ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from pathlib import Path
from datetime import datetime, timedelta
import asyncio
import time

# IBKR Integration
try:
    from ib_insync import *
    IBKR_AVAILABLE = True
    print("✅ ib_insync imported successfully")
except ImportError:
    IBKR_AVAILABLE = False
    print("❌ ib_insync not available - will use fallback data only")

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("📦 All imports completed successfully")
print(f"🕒 Notebook started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 🔌 IBKR Data Retrieval System

In [ ]:
class IBKRDataManager:
    """Manages IBKR TWS connection and data retrieval with robust error handling."""
    
    def __init__(self, host=TWS_HOST, port=TWS_PORT, client_id=CLIENT_ID):
        self.host = host
        self.port = port
        self.client_id = client_id
        self.ib = None
        self.connected = False
        
    def connect(self):
        """Establish connection to TWS/Gateway."""
        if not IBKR_AVAILABLE:
            print("❌ IBKR not available - cannot connect")
            return False
            
        try:
            print(f"🔌 Connecting to IBKR TWS at {self.host}:{self.port}...")
            self.ib = IB()
            self.ib.connect(self.host, self.port, clientId=self.client_id)
            self.connected = True
            print(f"✅ Connected successfully! Client ID: {self.client_id}")
            
            # Display connection info
            account_summary = self.ib.accountSummary()
            if account_summary:
                print(f"📊 Connected to account with {len(account_summary)} summary items")
            
            return True
            
        except Exception as e:
            print(f"❌ Failed to connect to IBKR: {e}")
            print("💡 Make sure TWS or IB Gateway is running and API is enabled")
            self.connected = False
            return False
    
    def create_contract(self, symbol, exchange=EXCHANGE, currency=CURRENCY):
        """Create IBKR contract for the specified symbol."""
        contract = Stock(symbol, exchange, currency)
        return contract
    
    def fetch_historical_data(self, symbol, duration=DATA_DURATION, bar_size=BAR_SIZE, what_to_show=WHAT_TO_SHOW):
        """Fetch historical data from IBKR."""
        if not self.connected:
            print("❌ Not connected to IBKR")
            return None
            
        try:
            print(f"📡 Fetching {duration} of {bar_size} data for {symbol}...")
            
            # Create contract
            contract = self.create_contract(symbol)
            
            # Qualify contract
            qualified_contracts = self.ib.qualifyContracts(contract)
            if not qualified_contracts:
                print(f"❌ Could not qualify contract for {symbol}")
                return None
                
            contract = qualified_contracts[0]
            print(f"✅ Contract qualified: {contract.symbol} ({contract.exchange})")
            
            # Request historical data
            bars = self.ib.reqHistoricalData(
                contract,
                endDateTime='',
                durationStr=duration,
                barSizeSetting=bar_size,
                whatToShow=what_to_show,
                useRTH=True,  # Regular trading hours only
                formatDate=1
            )
            
            if not bars:
                print(f"❌ No historical data received for {symbol}")
                return None
                
            print(f"✅ Received {len(bars)} bars for {symbol}")
            
            # Convert to DataFrame
            df = util.df(bars)
            
            # Standardize column names
            df = df.rename(columns={
                'date': 'datetime'
            })
            
            # Ensure datetime column
            if 'datetime' in df.columns:
                df['datetime'] = pd.to_datetime(df['datetime'])
            
            # Add date column for strategy
            df['date'] = df['datetime'].dt.date
            
            print(f"📊 Data range: {df['datetime'].min()} to {df['datetime'].max()}")
            print(f"💾 Sample data:\n{df.head(3)}")
            
            return df
            
        except Exception as e:
            print(f"❌ Error fetching data for {symbol}: {e}")
            return None
    
    def disconnect(self):
        """Safely disconnect from IBKR."""
        if self.ib and self.connected:
            try:
                self.ib.disconnect()
                print("🔌 Disconnected from IBKR")
            except:
                pass
            self.connected = False

# Initialize data manager
data_manager = IBKRDataManager()
print("🏗️ IBKR Data Manager initialized")

## 📊 Data Loading and Processing

In [ ]:
def load_fallback_data(csv_path):
    """Load data from CSV file as fallback."""
    try:
        print(f"📁 Loading fallback data from {csv_path}...")
        df = pd.read_csv(csv_path)
        
        # Normalize column names
        df.columns = [c.strip().lower() for c in df.columns]
        
        # Handle datetime column
        if "time" in df.columns and "datetime" not in df.columns:
            df.rename(columns={"time": "datetime"}, inplace=True)
        
        # Parse datetime
        if np.issubdtype(df["datetime"].dtype, np.number):
            if df["datetime"].max() > 1e12:
                df["datetime"] = pd.to_datetime(df["datetime"], unit="ms")
            else:
                df["datetime"] = pd.to_datetime(df["datetime"], unit="s")
        else:
            df["datetime"] = pd.to_datetime(df["datetime"])
        
        # Sort and process
        df = df.sort_values("datetime").reset_index(drop=True)
        df[["open","high","low","close"]] = df[["open","high","low","close"]].astype(float)
        df["date"] = df["datetime"].dt.date
        
        print(f"✅ Loaded {len(df)} rows from fallback CSV")
        return df
        
    except Exception as e:
        print(f"❌ Failed to load fallback data: {e}")
        return None

def get_data_for_ticker(ticker_symbol):
    """Get data for specified ticker, trying IBKR first, then fallback."""
    df = None
    data_source = "unknown"
    
    # Try IBKR first
    if IBKR_AVAILABLE and data_manager.connect():
        df = data_manager.fetch_historical_data(ticker_symbol)
        if df is not None:
            data_source = "IBKR_TWS"
            print(f"📊 Using IBKR data for {ticker_symbol}")
        data_manager.disconnect()
    
    # Try fallback if IBKR failed and ticker is QQQ
    if df is None and USE_FALLBACK_DATA:
        if ticker_symbol.upper() == "QQQ" and Path(FALLBACK_CSV_PATH).exists():
            df = load_fallback_data(FALLBACK_CSV_PATH)
            if df is not None:
                data_source = "CSV_Fallback"
                print(f"📁 Using CSV fallback data for {ticker_symbol}")
        else:
            print(f"⚠️ No fallback data available for {ticker_symbol}")
    
    if df is None:
        print(f"❌ Could not obtain data for {ticker_symbol}")
        return None, None
    
    # Validate data
    required_cols = {"datetime", "open", "high", "low", "close"}
    missing = required_cols - set(df.columns)
    if missing:
        print(f"❌ Missing required columns: {missing}")
        return None, None
    
    print(f"✅ Data validation passed for {ticker_symbol}")
    print(f"📊 Data source: {data_source}")
    print(f"📈 Price range: ${df['close'].min():.2f} - ${df['close'].max():.2f}")
    print(f"📅 Date range: {df['datetime'].min().date()} to {df['datetime'].max().date()}")
    
    return df, data_source

# Load data for configured ticker
print(f"🚀 Loading data for {TICKER_SYMBOL}...")
print("=" * 60)

df, data_source = get_data_for_ticker(TICKER_SYMBOL)

if df is not None:
    print(f"\n✅ Successfully loaded {len(df):,} bars for {TICKER_SYMBOL}")
    print(f"📊 Sample data:")
    print(df.head(3).to_string(index=False))
else:
    print(f"\n❌ Failed to load data for {TICKER_SYMBOL}")
    print("💡 Please check:")
    print("   • IBKR TWS/Gateway is running")
    print("   • API connections are enabled in TWS")
    print("   • Ticker symbol is correct")
    print("   • Market data subscriptions are active")

## 🎯 Enhanced Strategy Engine

In [ ]:
def compute_sma(series: pd.Series, period: int) -> pd.Series:
    """Compute Simple Moving Average."""
    if period and period > 0:
        return series.rolling(period, min_periods=period).mean()
    return pd.Series([np.nan]*len(series), index=series.index)

def first_bar_each_day(frame: pd.DataFrame) -> pd.DataFrame:
    """Return the first bar of each calendar date."""
    return frame.groupby("date", as_index=False).head(1).copy()

def exit_same_day(frame: pd.DataFrame, entry_idx: int, side: str, profit_target: float):
    """Determine exit time/price for same-day trade with profit target."""
    entry_row = frame.loc[entry_idx]
    entry_price = entry_row["close"]
    entry_date = entry_row["date"]

    # Get bars of the same day after entry
    day_df = frame.loc[frame["date"].eq(entry_date)]
    after = day_df[day_df["datetime"] > entry_row["datetime"]]

    # Check for profit target hit
    if profit_target and profit_target > 0:
        if side == "buy":
            target_price = entry_price * (1 + profit_target / 100.0)
            hit_bars = after[after["high"] >= target_price]
            if len(hit_bars):
                return hit_bars.iloc[0]["datetime"], float(target_price)
        else:  # sell
            target_price = entry_price * (1 - profit_target / 100.0)
            hit_bars = after[after["low"] <= target_price]
            if len(hit_bars):
                return hit_bars.iloc[0]["datetime"], float(target_price)

    # Fallback: exit at end of day
    last_bar = day_df.iloc[-1]
    return last_bar["datetime"], float(last_bar["close"])

def run_strategy(
    data: pd.DataFrame,
    ticker_symbol: str,
    trade_mode: str = "both",
    sma_period: int = 0,
    profit_target: float = 0.0
):
    """Run First Bar Strategy with enhanced multi-ticker support."""
    
    if data is None or data.empty:
        return None, None, None
    
    print(f"🎯 Running First Bar Strategy for {ticker_symbol}...")
    print(f"⚙️ Parameters: Mode={trade_mode}, SMA={sma_period}, Target={profit_target}%")
    
    frame = data.copy()
    frame["sma"] = compute_sma(frame["close"], sma_period)
    firsts = first_bar_each_day(frame)
    
    print(f"📊 Found {len(firsts)} first bars to analyze")
    
    trades = []
    for _, row in firsts.iterrows():
        o, c = row["open"], row["close"]
        idx = row.name

        # Generate signal
        side = None
        if o < c: 
            side = "buy"
        elif o > c: 
            side = "sell"

        if not side:
            continue

        # Apply SMA filter
        if sma_period and not np.isnan(row["sma"]):
            if row["close"] > row["sma"] and side != "buy":
                continue
            elif row["close"] < row["sma"] and side != "sell":
                continue
            elif row["close"] == row["sma"]:
                continue

        # Apply trade mode filter
        if trade_mode == "buy" and side != "buy":
            continue
        if trade_mode == "sell" and side != "sell":
            continue

        entry_time = row["datetime"]
        entry_price = row["close"]

        exit_time, exit_price = exit_same_day(frame, idx, side, profit_target)

        # Calculate P&L in points
        pnl = (exit_price - entry_price) if side == "buy" else (entry_price - exit_price)
        
        # Calculate percentage return
        pnl_pct = (pnl / entry_price) * 100

        trades.append({
            "ticker": ticker_symbol,
            "side": side,
            "date": row["date"],
            "entry_time": entry_time,
            "exit_time": exit_time,
            "entry": float(entry_price),
            "exit": float(exit_price),
            "pnl": float(pnl),
            "pnl_pct": float(pnl_pct)
        })

    # Create trades DataFrame
    trades_df = pd.DataFrame(trades)
    if trades_df.empty:
        print("❌ No trades generated with current parameters")
        return pd.DataFrame(), {}, pd.DataFrame()

    trades_df = trades_df.sort_values("exit_time").reset_index(drop=True)
    trades_df["cum_pnl"] = trades_df["pnl"].cumsum()
    trades_df["cum_pnl_pct"] = trades_df["pnl_pct"].cumsum()

    # Create equity curve
    equity = trades_df[["exit_time","cum_pnl","cum_pnl_pct"]].rename(
        columns={"exit_time":"time","cum_pnl":"equity","cum_pnl_pct":"equity_pct"}
    ).copy()
    equity["peak"] = equity["equity"].cummax()
    equity["drawdown"] = equity["equity"] - equity["peak"]
    equity["peak_pct"] = equity["equity_pct"].cummax()
    equity["drawdown_pct"] = equity["equity_pct"] - equity["peak_pct"]
    
    max_dd = float(equity["drawdown"].min())
    max_dd_pct = float(equity["drawdown_pct"].min())

    # Calculate statistics by side
    def side_stats(s: str):
        sdf = trades_df[trades_df["side"] == s]
        if sdf.empty:
            return {
                "count": 0, "total_pnl": 0.0, "total_pnl_pct": 0.0,
                "max_winner": 0.0, "max_loser": 0.0,
                "avg_winner": 0.0, "avg_loser": 0.0,
                "win_rate_pct": 0.0
            }
        
        wins = sdf[sdf["pnl"] > 0]
        losses = sdf[sdf["pnl"] < 0]
        
        return {
            "count": len(sdf),
            "total_pnl": float(sdf["pnl"].sum()),
            "total_pnl_pct": float(sdf["pnl_pct"].sum()),
            "max_winner": float(wins["pnl"].max()) if len(wins) > 0 else 0.0,
            "max_loser": float(losses["pnl"].min()) if len(losses) > 0 else 0.0,
            "avg_winner": float(wins["pnl"].mean()) if len(wins) > 0 else 0.0,
            "avg_loser": float(losses["pnl"].mean()) if len(losses) > 0 else 0.0,
            "win_rate_pct": float((len(wins) / len(sdf)) * 100) if len(sdf) > 0 else 0.0
        }

    # Compile summary statistics
    summary = {
        "ticker": ticker_symbol,
        "total_trades": len(trades_df),
        "buy": side_stats("buy"),
        "sell": side_stats("sell"),
        "all": side_stats(""),  # All trades
        "max_drawdown": float(max_dd),
        "max_drawdown_pct": float(max_dd_pct),
        "data_source": data_source
    }
    
    # Update "all" stats manually since side filter doesn't work
    all_wins = trades_df[trades_df["pnl"] > 0]
    all_losses = trades_df[trades_df["pnl"] < 0]
    
    summary["all"] = {
        "count": len(trades_df),
        "total_pnl": float(trades_df["pnl"].sum()),
        "total_pnl_pct": float(trades_df["pnl_pct"].sum()),
        "max_winner": float(all_wins["pnl"].max()) if len(all_wins) > 0 else 0.0,
        "max_loser": float(all_losses["pnl"].min()) if len(all_losses) > 0 else 0.0,
        "avg_winner": float(all_wins["pnl"].mean()) if len(all_wins) > 0 else 0.0,
        "avg_loser": float(all_losses["pnl"].mean()) if len(all_losses) > 0 else 0.0,
        "win_rate_pct": float((len(all_wins) / len(trades_df)) * 100) if len(trades_df) > 0 else 0.0
    }
    
    print(f"✅ Strategy completed: {len(trades_df):,} trades generated")
    return trades_df, summary, equity

print("🎯 Enhanced Strategy Engine loaded")

## 🚀 Execute Strategy

In [ ]:
# Only run if we have data
if df is not None:
    print(f"🚀 Executing {TICKER_SYMBOL} First Bar Strategy...")
    print("=" * 60)
    
    # Run the strategy
    trades_df, summary, equity = run_strategy(
        df,
        TICKER_SYMBOL,
        trade_mode=TRADE_MODE,
        sma_period=SMA_PERIOD,
        profit_target=PROFIT_TARGET
    )
    
    if not trades_df.empty:
        print(f"\n{'='*70}")
        print(f"🏆 {TICKER_SYMBOL} FIRST BAR STRATEGY RESULTS")
        print(f"{'='*70}")
        
        # Display key metrics
        total_trades = summary['total_trades']
        total_pnl = summary['all']['total_pnl']
        total_pnl_pct = summary['all']['total_pnl_pct']
        win_rate = summary['all']['win_rate_pct']
        max_dd = summary['max_drawdown']
        max_dd_pct = summary['max_drawdown_pct']
        
        print(f"📊 Total Trades: {total_trades:,}")
        print(f"💰 Total P&L: ${total_pnl:,.2f} ({total_pnl_pct:+.2f}%)")
        print(f"🎯 Win Rate: {win_rate:.1f}%")
        print(f"📉 Max Drawdown: ${max_dd:.2f} ({max_dd_pct:.2f}%)")
        
        # Calculate risk-adjusted return
        if abs(max_dd) > 0:
            risk_adj_ratio = abs(total_pnl / max_dd)
            print(f"🏆 Risk-Adjusted Ratio: {risk_adj_ratio:.2f}")
        
        print(f"📊 Data Source: {summary['data_source']}")
        
        # Show first few trades
        print(f"\n📋 Sample Trades (First 5):")
        print(trades_df[['side','date','entry','exit','pnl','pnl_pct']].head().to_string(index=False))
        
    else:
        print("❌ No trades were generated")
else:
    print("❌ Cannot execute strategy - no data available")

## 📊 Performance Analysis and Visualizations

In [ ]:
if df is not None and not trades_df.empty:
    print(f"📊 Creating performance analysis for {TICKER_SYMBOL}...")
    
    # Create comprehensive visualization dashboard
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(f'{TICKER_SYMBOL} First Bar Strategy - Performance Dashboard', fontsize=16, fontweight='bold')
    
    # 1. Equity Curve (in dollars)
    ax1.plot(equity["time"], equity["equity"], linewidth=2, color='green', label='Strategy P&L')
    ax1.fill_between(equity["time"], equity["equity"], 0, alpha=0.3, color='green')
    ax1.set_title(f'💰 Cumulative P&L - ${equity["equity"].iloc[-1]:,.2f}', fontweight='bold')
    ax1.set_ylabel('P&L ($)')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # 2. Drawdown Chart
    ax2.fill_between(equity["time"], equity["drawdown"], 0, color='red', alpha=0.3)
    ax2.plot(equity["time"], equity["drawdown"], color='red', linewidth=1)
    ax2.set_title(f'📉 Drawdown - Max: ${summary["max_drawdown"]:.2f}', fontweight='bold')
    ax2.set_ylabel('Drawdown ($)')
    ax2.grid(True, alpha=0.3)
    
    # 3. Trade Distribution
    winners = trades_df[trades_df["pnl"] > 0]
    losers = trades_df[trades_df["pnl"] < 0]
    
    if len(winners) > 0:
        ax3.hist(winners["pnl"], bins=20, alpha=0.7, color='green', label=f'Winners ({len(winners)})', edgecolor='black')
    if len(losers) > 0:
        ax3.hist(losers["pnl"], bins=20, alpha=0.7, color='red', label=f'Losers ({len(losers)})', edgecolor='black')
    
    ax3.set_title('📊 Trade P&L Distribution', fontweight='bold')
    ax3.set_xlabel('P&L per Trade ($)')
    ax3.set_ylabel('Frequency')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Monthly Returns Heatmap (simplified)
    trades_monthly = trades_df.copy()
    trades_monthly['month'] = pd.to_datetime(trades_monthly['date']).dt.to_period('M')
    monthly_pnl = trades_monthly.groupby('month')['pnl'].sum()
    
    if len(monthly_pnl) > 0:
        # Plot monthly returns as bar chart
        colors = ['green' if x > 0 else 'red' for x in monthly_pnl.values]
        bars = ax4.bar(range(len(monthly_pnl)), monthly_pnl.values, color=colors, alpha=0.7, edgecolor='black')
        ax4.set_title('📅 Monthly Returns', fontweight='bold')
        ax4.set_ylabel('Monthly P&L ($)')
        ax4.set_xlabel('Month')
        ax4.grid(True, alpha=0.3, axis='y')
        
        # Limit x-axis labels to avoid crowding
        if len(monthly_pnl) > 12:
            step = max(1, len(monthly_pnl) // 10)
            ax4.set_xticks(range(0, len(monthly_pnl), step))
            ax4.set_xticklabels([str(monthly_pnl.index[i]) for i in range(0, len(monthly_pnl), step)], rotation=45)
        else:
            ax4.set_xticks(range(len(monthly_pnl)))
            ax4.set_xticklabels([str(x) for x in monthly_pnl.index], rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Additional Statistics Table
    print(f"\n{'='*70}")
    print(f"📈 DETAILED PERFORMANCE METRICS FOR {TICKER_SYMBOL}")
    print(f"{'='*70}")
    
    def print_side_stats(side_name, stats):
        if stats['count'] > 0:
            print(f"\n🎯 {side_name.upper()} TRADES:")
            print(f"   • Count: {stats['count']:,}")
            print(f"   • Total P&L: ${stats['total_pnl']:,.2f}")
            print(f"   • Win Rate: {stats['win_rate_pct']:.1f}%")
            print(f"   • Avg Winner: ${stats['avg_winner']:.2f}")
            print(f"   • Avg Loser: ${stats['avg_loser']:.2f}")
            print(f"   • Max Winner: ${stats['max_winner']:.2f}")
            print(f"   • Max Loser: ${stats['max_loser']:.2f}")
    
    print_side_stats("BUY", summary['buy'])
    print_side_stats("SELL", summary['sell'])
    print_side_stats("OVERALL", summary['all'])
    
    # Export results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    trades_file = f"{TICKER_SYMBOL.lower()}_strategy_trades_{timestamp}.csv"
    equity_file = f"{TICKER_SYMBOL.lower()}_strategy_equity_{timestamp}.csv"
    
    trades_df.to_csv(trades_file, index=False)
    equity.to_csv(equity_file, index=False)
    
    print(f"\n💾 Results exported:")
    print(f"   • Trades: {trades_file}")
    print(f"   • Equity: {equity_file}")
    
else:
    print("⚠️ Skipping analysis - no data or trades available")

## 🔄 Quick Ticker Comparison

**To test different tickers:**
1. Go back to the **Configuration Section**
2. Change `TICKER_SYMBOL = "TQQQ"` to your desired ticker
3. Run all cells again

**Suggested tickers to test:**
- `"QQQ"` - Nasdaq-100 ETF
- `"TQQQ"` - 3x Leveraged Nasdaq-100 ETF
- `"SPY"` - S&P 500 ETF
- `"IWM"` - Russell 2000 ETF
- `"DIA"` - Dow Jones ETF

## ⚙️ Parameter Optimization (Optional)

Run this section to optimize profit targets for the current ticker:

In [ ]:
# Uncomment and run this cell to perform parameter optimization

# if df is not None:
#     print(f"🔧 Starting parameter optimization for {TICKER_SYMBOL}...")
#     
#     trade_modes = ["buy", "sell", "both"]
#     profit_targets = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]
#     
#     results = []
#     total_runs = len(trade_modes) * len(profit_targets)
#     run_count = 0
#     
#     print(f"📊 Testing {total_runs} parameter combinations...")
#     
#     for trade_mode in trade_modes:
#         for profit_target in profit_targets:
#             run_count += 1
#             
#             trades, summary, equity = run_strategy(
#                 df, TICKER_SYMBOL, trade_mode=trade_mode, 
#                 sma_period=0, profit_target=profit_target
#             )
#             
#             if not trades.empty:
#                 total_pnl = summary['all']['total_pnl']
#                 max_dd = abs(summary['max_drawdown'])
#                 ratio = total_pnl / max_dd if max_dd > 0 else 0
#                 
#                 results.append({
#                     'ticker': TICKER_SYMBOL,
#                     'trade_mode': trade_mode,
#                     'profit_target': profit_target,
#                     'trades': summary['total_trades'],
#                     'total_pnl': total_pnl,
#                     'max_drawdown': -max_dd,
#                     'pnl_dd_ratio': ratio,
#                     'win_rate': summary['all']['win_rate_pct']
#                 })
#             
#             if run_count % 5 == 0:
#                 print(f"⏳ Progress: {run_count}/{total_runs} ({100*run_count/total_runs:.0f}%)")
#     
#     # Display results
#     if results:
#         opt_df = pd.DataFrame(results).sort_values('pnl_dd_ratio', ascending=False)
#         
#         print(f"\n🏆 TOP 10 OPTIMIZATION RESULTS FOR {TICKER_SYMBOL}:")
#         print(opt_df.head(10).to_string(index=False))
#         
#         # Export optimization results
#         opt_file = f"{TICKER_SYMBOL.lower()}_optimization_results.csv"
#         opt_df.to_csv(opt_file, index=False)
#         print(f"\n💾 Optimization results exported: {opt_file}")
# else:
#     print("❌ Cannot run optimization - no data available")

print("💡 Uncomment the code above to run parameter optimization")

## 📋 Strategy Summary

### Key Results:
- **Ticker Tested:** {TICKER_SYMBOL}
- **Strategy:** First Bar momentum trading
- **Data Source:** IBKR TWS API with CSV fallback
- **Flexibility:** Easy ticker switching via configuration

### Next Steps:
1. **Test other tickers** by changing `TICKER_SYMBOL` in the configuration
2. **Run optimization** to find best parameters for each ticker
3. **Compare results** across different assets
4. **Implement risk management** based on ticker volatility

### Disclaimers:
- This is for educational and research purposes only
- Past performance does not guarantee future results
- TQQQ and leveraged ETFs carry additional risks
- Always validate with paper trading before live implementation

---

**🤖 Multi-Ticker First Bar Strategy - Powered by IBKR TWS API**